# Edit Knowledge Graphs

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li>
        <li><span><a href="#Data-Editing" data-toc-modified-id="Data-Editing-2">Data Editing</a></span>
            <ul>
                <li><span><a href="#Adds" data-toc-modified-id="Adds-2.1">Adds</a></span></li>
                <li><span><a href="#Updates" data-toc-modified-id="Updates-2.2">Updates</a></span></li>
                <li><span><a href="#Deletes" data-toc-modified-id="Deletes-2.3">Deletes</a></span></li>
            </ul>
        </li>
        <li><span><a href="#Data-Model-Editing" data-toc-modified-id="Data-Model-Editing-3">Data Model Editing</a></span>
            <ul>
                <li><span><a href="#Types" data-toc-modified-id="Types-3.1">Types</a></span>
                    <ul>
                        <li><span><a href="#Adds" data-toc-modified-id="Adds-3.1.1">Adds</a></span></li>
                        <li><span><a href="#Updates" data-toc-modified-id="Updates-3.1.2">Updates</a></span></li>
                        <li><span><a href="#Deletes" data-toc-modified-id="Deletes-3.1.3">Deletes</a></span></li>
                    </ul>
                </li>
                <li><span><a href="#Properties" data-toc-modified-id="Properties-3.2">Properties</a></span></li>
                    <ul>
                        <li><span><a href="#Adds" data-toc-modified-id="Adds-3.2.1">Adds</a></span></li>
                        <li><span><a href="#Updates" data-toc-modified-id="Updates-3.2.2">Updates</a></span></li>
                        <li><span><a href="#Deletes" data-toc-modified-id="Deletes-3.2.3">Deletes</a></span></li>
                    </ul>
                </li>
                <li><span><a href="#Search-Indexes" data-toc-modified-id="Search-Indexes-3.3">Search Indexes</a></span></li>
            </ul>
        </li>
    </ul>
</div>

## Introduction

The ArcGIS API for Python allows editing of both the data and data model of the `KnowledgeGraph`. The goal of this guide is to provide information and examples of how to edit your knowledge graphs using python.

Editing the data will allow for:
- Creating and deleting entities
- Creating and deleting relationships
- Updating property values

Editing the data model will allow for:
- Creating and deleting entity types
- Creating and deleting relationship types
- Creating and deleting properties

## Data Editing
All data editing is accomplished using [knowledge_graph.apply_edits()](https://developers.arcgis.com/python/api-reference/arcgis.graph.html#arcgis.graph.KnowledgeGraph.apply_edits)
### Adds

To add an entity to the knowledge graph, we define the type and properties of the entity:

In [ ]:
add_entity = Entity(
    type_name="Company", 
    properties={
        "name": "Esri",
        "year_established": 1969
    }
)

knowledge_graph.apply_edits(adds=[add_entity], as_dict=False)

To add a relationship to the knowledge graph we must know which two entities we want to create the relationship between and use their ids along with information about the relationship (in the form of properties) to create it:

In [ ]:
add_relationship = Relationship(
    type_name="WorksFor", 
    origin_entity_id="{56BCCAF0-DD03-487A-BE39-F32164714190}",
    destination_entity_id="{26BCCAF0-DD03-484A-BE39-B32164718190}",
    properties={
        "startDate": datetime(2020, 1, 6)
    }
)

knowledge_graph.apply_edits(adds=[add_relationship], as_dict=False)

### Updates

To update the value of a property on an entity, provide the id of the entity along with the properties and values that you would like updated. This can include geometries for spatial entities:

In [ ]:
update_entity = Entity(
    type_name="Company", 
    id="{26BCCAF0-DD03-484A-BE39-B32164718190}",
    properties={
        "name": "Not Esri",
        "shape": Point(
            {
                "x":-89.86140978600001, 
                "y":38.902491682000004,
                "spatialReference": {"wkid": 4326}
            }
        )
    }
)

knowledge_graph.apply_edits(updates=[update_entity], as_dict=False)

### Deletes

To delete instances of a type, the ids to be deleted need to be defined with the name of the type:

In [ ]:
delete_entity = EntityDelete(
    type_name="Company", 
    ids=["{26BCCAF0-DD03-484A-BE39-B32164718190}"]
)

knowledge_graph.apply_edits(deletes=[delete_entity], cascade_delete=True, as_dict=False) # be sure to set cascade_delete to True to automatically delete relationships attached to the provided entity

## Data Model Editing

### Types

#### Adds

Adding new types involves forming the type objects and pushing them into the data model of the knowledge graph using [knowledge_graph.named_object_type_adds()](https://developers.arcgis.com/python/api-reference/arcgis.graph.html#arcgis.graph.KnowledgeGraph.named_object_type_adds). Named types (entity and relationship types) can be created in bulk or individually as needed. The following example shows creating these named types all at the same time:

In [ ]:
# create the data model
knowledge_graph.named_object_type_adds(
    entity_types=[
        EntityType(name="User", properties={"name": GraphProperty(name="name")}),
        EntityType(name="Item", properties={
            "id": GraphProperty(name="id"),
            "title": GraphProperty(name="title"),
            "type": GraphProperty(name="type")
        })
    ], 
    relationship_types=[
        RelationshipType(name="DependsOn"),
        RelationshipType(name="Owns")
    ],
    as_dict=False
)

#### Updates

Updates are performed on named object types using [knowledge_graph.named_object_type_update()](https://developers.arcgis.com/python/api-reference/arcgis.graph.html#arcgis.graph.KnowledgeGraph.named_object_type_update). Updating the name of a type is not allowed.

In [ ]:
knowledge_graph.named_object_type_update(
    type_name="User", 
    named_type_update=EntityType(name="User", alias="NewUser", strict=False), 
    mask=NamedObjectTypeMask(
        update_alias=True,
        update_strict=True
    ),
    as_dict=False
)

#### Deletes

Deleting types is as simple as passing the string name of the type to [knowledge_graph.named_object_type_delete()](https://developers.arcgis.com/python/api-reference/arcgis.graph.html#arcgis.graph.KnowledgeGraph.named_object_type_delete)

In [ ]:
knowledge_graph.named_object_type_delete(type_name="User", as_dict=False)

### Properties

#### Adds

Properties can be added to named types at any point using [knowledge_graph.graph_property_adds()](https://developers.arcgis.com/python/api-reference/arcgis.graph.html#arcgis.graph.KnowledgeGraph.graph_property_adds). Many properties can be added to a given type in a single call.

The default type for a property is esriFieldTypeString, to define a different type use one of the [available field types](https://github.com/Esri/knowledge-pbf/blob/master/proto/esriPBuffer/EsriExtendedTypes/EsriExtendedTypes.proto):
- esriFieldTypeSmallInteger
- esriFieldTypeBigInteger
- esriFieldTypeInteger
- esriFieldTypeSingle
- esriFieldTypeDouble
- esriFieldTypeString
- esriFieldTypeDate
- esriFieldTypeDateOnly
- esriFieldTypeTimeOnly
- esriFieldTypeTimestampOffset
- esriFieldTypeOID
- esriFieldTypeGeometry
    - with this type, you also need to [define geometryType](https://github.com/Esri/knowledge-pbf/blob/master/proto/esriPBuffer/EsriTypes.proto):
        - esriGeometryPoint
        - esriGeometryPolyline
        - esriGeometryPolygon
        - esriGeometryMultipoint
- esriFieldTypeGUID
- esriFieldTypeGlobalID


In [ ]:
knowledge_graph.graph_property_adds(
    type_name="User", 
    graph_properties=[
        GraphProperty(name="name"),
        Graphproperty(name="post_count", field_type="esriFieldTypeInteger"),
        GraphProperty(name="shape", field_type="esriFieldTypeGeometry", geometry_type="esriGeometryPoint")
    ]
)

#### Updates

Information about the properties on a type can be updated using [knowledge_graph.graph_property_update()](https://developers.arcgis.com/python/api-reference/arcgis.graph.html#arcgis.graph.KnowledgeGraph.graph_property_update). Only one property of a given type can be updated in a single call.

In [ ]:
knowledge_graph.graph_property_update(
    type_name="User", 
    property_name="name",
    graph_property=GraphProperty(name="name", alias="user_name", field_type="esriFieldTypeString"),
    mask=GraphPropertyMask(
        update_alias=True,
        update_field_type=True
    ),
    as_dict=False
)

#### Deletes

Deleting a property is a very simple, just pass the name of the type and property to be deleted using [knowledge_graph.graph_property_delete()](https://developers.arcgis.com/python/api-reference/arcgis.graph.html#arcgis.graph.KnowledgeGraph.graph_property_delete). Only one property from any given type can be deleted in a single call. To delete multiple properties, it may be good to loop through a list of properties you plan to delete or make multiple calls consecutively.

In [ ]:
graph.graph_property_delete(type_name="User", property_name="name", as_dict=False)

### Search Indexes

Search indexes can be added to esriFieldTypeString properties to allow the contents of those properties to be searched for when using [knowledge_graph.search()](https://developers.arcgis.com/python/api-reference/arcgis.graph.html#arcgis.graph.KnowledgeGraph.search).

In [ ]:
# add search indexes to individual properties
knowledge_graph.update_search_index(adds={"User": SearchIndexProperties(property_names=["name"])}, as_dict=False)

# add search indexes to all text properties based on the data model
datamodel = knowledge_graph.query_data_model()
for entity_type in datamodel.entity_types:
    prop_list = []
    for prop in datamodel.entity_types[entity_type].properties:
        if datamodel.entity_types[entity_type].properties[prop].fieldType == 'esriFieldTypeString':
            prop_list.append(prop)
    knowledge_graph.update_search_index(adds={entity_type: SearchIndexProperties(property_names=prop_list)}, as_dict=False)
for entity_type in datamodel.relationship_types:
    prop_list = []
    for prop in datamodel.relationship_types[entity_type].properties:
        if datamodel.relationship_types[entity_type].properties[prop].fieldType == 'esriFieldTypeString':
            prop_list.append(prop)
    knowledge_graph.update_search_index(adds={entity_type: SearchIndexProperties(property_names=prop_list)}, as_dict=False)

# delete a property from the search index
knowledge_graph.update_search_index(deletes={"User": SearchIndexProperties(property_names=["name"])}, as_dict=False)